In [1]:
!cp '../input/quanvh8-code/quanvh8_funcs.py' .

In [2]:
'''ENSEMBLE NETS
Inspire by https://www.kaggle.com/c/otto-group-product-classification-challenge/discussion/14335'''

import numpy as np, pandas as pd, copy, tensorflow as tf, matplotlib.pyplot as plt

from tensorflow import feature_column as fc
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.layers import (Dense, DenseFeatures, Dropout, 
                                     BatchNormalization, Embedding, Input, Concatenate, Average,
                                     InputLayer, Lambda)
from tensorflow.keras.metrics import AUC, Precision, Recall
from tensorflow.keras import backend as K, Sequential, Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam, RMSprop

import tensorflow_addons as tfa
from tensorflow_addons.layers import WeightNormalization

from sklearn.preprocessing import QuantileTransformer, StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.cluster import KMeans

from quanvh8_funcs import (DerivedFeatures)#, kfolds_bagging_training, voting_predict,
#                            kolds_stacked_ensemble_training, stacked_ensemble_predict )

import sys

def log_loss_metric(y_true, y_pred):
    bce = tf.keras.losses.BinaryCrossentropy()
    return bce(y_true, y_pred).numpy()

print(pd.__version__)
print(tf.__version__)

1.1.4
2.3.1


In [3]:
# Loading data and encoding

folder_path = '../input/lish-moa/'
raw_test = pd.read_csv(folder_path + 'test_features.csv')
raw_train = pd.read_csv(folder_path + 'train_features.csv')
raw_targets = pd.read_csv(folder_path + 'train_targets_scored.csv')

# Phân loại dữ liệu
cols_id = ['sig_id']
cols_to_remove = ['cp_type']
cols_fts = [i for i in raw_train.columns if i not in cols_id +cols_to_remove]
cols_gene = [col for col in raw_train.columns if col.startswith("g-")]
cols_cell = [col for col in raw_train.columns if col.startswith("c-")]
cols_experiment = [col for col in cols_fts if col not in cols_gene+cols_cell]
cols_target = [i for i in raw_targets.columns if i not in cols_id]
num_fts, num_labels = len(cols_fts), len(cols_target)

# xử lý categorical
def transform_data(input_data):
    '''Clean data and encoding
        * input_data: table '''
    out = input_data.copy()
    out['cp_dose'] = out['cp_dose'].map({'D1':0, 'D2':1})
    out['cp_time'] = out['cp_time']/72
    
    return out

to_train = transform_data(raw_train[raw_train['cp_type'] != 'ctl_vehicle'])
to_train_targets = raw_targets.iloc[to_train.index]
full_pred  = transform_data(raw_test)
to_pred = full_pred[full_pred['cp_type'] != 'ctl_vehicle']

In [4]:
# preprocessing pipeline
def pipe_line_builder(quantiles_num, pca_dims, kmean_clusters):
    '''Dựng pipe line cho từng nhóm columns
    :quantiles_num: int: số quantile khi normalise
    :pca_dims: int: số chiều pca'''
    norm = QuantileTransformer(n_quantiles=quantiles_num,random_state=0, output_distribution="normal")
    pca = PCA(n_components = pca_dims)
    derived_ft = DerivedFeatures(n_clusters = kmean_clusters)

    p_derived_ft = Pipeline([
        ('norm', norm), 
        ('derived', derived_ft)])

    p_norm_pca = Pipeline([ 
        ('norm', norm),
        ('pca', pca) ])
    return FeatureUnion([
        ('norm', norm), 
        ('norm_pca', p_norm_pca),
        ('derived', p_derived_ft)])

# 

pipe = Pipeline([
    ('norm_pca', ColumnTransformer([
                     ('gene', pipe_line_builder(quantiles_num = 200, pca_dims = 600, kmean_clusters = 5), cols_gene),
                     ('cell', pipe_line_builder(quantiles_num = 200, pca_dims = 50, kmean_clusters = 5), cols_cell),
                    ]) 
    ), 
    ('var', VarianceThreshold(0.5)) 
])

pipe = ColumnTransformer([
    ('gene_cell', pipe, cols_gene+ cols_cell),
    ('experiment', 'passthrough', cols_experiment)
])

In [5]:
# Transform data
pipe.fit(to_train[cols_fts].append(to_pred[cols_fts]))
X_train = pipe.transform(to_train[cols_fts])
X_pred = pipe.transform(to_pred[cols_fts])
y_train = to_train_targets[cols_target].values

In [6]:
# Hyper params
NFOLDS = 8
BATCH_SIZE = 128
EPOCHS = 150
BAGGING_ALPHA = 0.75
SEEDS = [23, 228, 1488, 1998, 2208, 2077, 404]
KFOLDS = 10
EARLY_STOPPING_STEPS = 10
WEIGHT_DECAY = 1e-5
label_smoothing_alpha = 0.001
P_MIN = label_smoothing_alpha
P_MAX = 1 - P_MIN

In [17]:
# Define model
model = Sequential([
    BatchNormalization(),
    WeightNormalization(Dense(1024 )),
    tf.keras.layers.PReLU(),

    BatchNormalization(),
    Dropout(0.25),
    WeightNormalization(Dense(1024)),
    tf.keras.layers.PReLU(),
    BatchNormalization(),
    Dropout(0.25),
    WeightNormalization(Dense(512)),
    tf.keras.layers.PReLU(),
    BatchNormalization(),
    Dropout(0.25),
    WeightNormalization(Dense(num_labels, activation="sigmoid"))
])

bce = tf.keras.losses.BinaryCrossentropy() #val_binary_crossentropy
model.compile(optimizer='adam', loss= BinaryCrossentropy(label_smoothing= label_smoothing_alpha)
              , metrics= [bce])

In [18]:
reduce_lr = ReduceLROnPlateau(monitor='val_binary_crossentropy', factor=0.3, patience=5, mode='min', min_lr=1E-5, verbose=1)
early_stopping = EarlyStopping(monitor='val_binary_crossentropy', min_delta=1E-5, patience=15, mode='min',restore_best_weights=True, verbose=1)
    
model.fit(
        X_train, y_train, validation_split = 0.3, 
        callbacks=[reduce_lr, early_stopping], epochs=150, verbose =1,
        batch_size=128)

Epoch 1/150
121/121 [==============================] - 9s 72ms/step - loss: 0.4816 - binary_crossentropy: 0.4784 - val_loss: 0.1107 - val_binary_crossentropy: 0.1107
Epoch 2/150
121/121 [==============================] - 8s 67ms/step - loss: 0.0605 - binary_crossentropy: 0.0602 - val_loss: 0.0326 - val_binary_crossentropy: 0.0325
Epoch 3/150
121/121 [==============================] - 8s 69ms/step - loss: 0.0289 - binary_crossentropy: 0.0288 - val_loss: 0.0241 - val_binary_crossentropy: 0.0239
Epoch 4/150
121/121 [==============================] - 8s 70ms/step - loss: 0.0235 - binary_crossentropy: 0.0233 - val_loss: 0.0217 - val_binary_crossentropy: 0.0215
Epoch 5/150
121/121 [==============================] - 8s 68ms/step - loss: 0.0211 - binary_crossentropy: 0.0209 - val_loss: 0.0199 - val_binary_crossentropy: 0.0198
Epoch 6/150
121/121 [==============================] - 8s 69ms/step - loss: 0.0198 - binary_crossentropy: 0.0200 - val_loss: 0.0197 - val_binary_crossentropy: 0.0195
Epoc

Un-Use code

In [ ]:
# # Define model
# model = Sequential([
#     # BatchNormalization(),
#     WeightNormalization(Dense(1024, activation="selu", kernel_regularizer= tf.keras.regularizers.l2(0.0005))),

#     # BatchNormalization(),
#     Dropout(0.25),
#     WeightNormalization(Dense(1024, activation="selu", kernel_regularizer= tf.keras.regularizers.l2(0.0005))),
#     # BatchNormalization(),
#     Dropout(0.25),
#     WeightNormalization(Dense(512, activation="selu", kernel_regularizer= tf.keras.regularizers.l2(0.0005))),
#     # BatchNormalization(),
#     Dropout(0.25),
#     WeightNormalization(Dense(num_labels, activation="sigmoid"))
# ])

# bce = tf.keras.losses.BinaryCrossentropy() #val_binary_crossentropy
# model.compile(optimizer='adam', loss= BinaryCrossentropy(label_smoothing= label_smoothing_alpha)
#               , metrics= [bce])